In [ ]:
import matplotlib.pyplot as plt
from quantopian.research.experimental import continuous_future, history
import pandas as pd
import numpy as np

### 5限月分の終値を取得

In [ ]:
def get_fut_hist(symbol, start, end):
    df = history(
        [continuous_future(symbol, offset=i,  adjustment=None) for i in range(5)],
        fields='price', 
        frequency='daily', 
        start=start, 
        end=end
    )

    df.columns = ['f{}'.format(i) for i in range(1, 6)]
    return df

In [ ]:
cl = get_fut_hist('CL', '2015-01-01', '2018-6-15')
ho = get_fut_hist('HO', '2015-01-01', '2018-6-15')
xb = get_fut_hist('XB', '2015-01-01', '2018-6-15')
ng = get_fut_hist('ng', '2015-01-01', '2018-6-15')

In [ ]:
cl.head()

### 満期日を追加


In [ ]:

def get_expiration_date(contract):
      return contract.expiration_date

def insert_expiration_date(symbol, start, end ):
    contract = history(continuous_future(symbol), fields=['contract'],frequency="daily", start=start, end=end)
    contract["expiration_date"] = contract['contract'].apply(get_expiration_date)
    contract["date"] = contract.index # memo1
    contract["remain_date"] = (contract["expiration_date"] - contract["date"]).dt.days
    return contract
    
cl_contract = insert_expiration_date('CL', '2015-01-01', '2018-6-15')
ho_contract = insert_expiration_date('HO', '2015-01-01', '2018-6-15')
xb_contract = insert_expiration_date('XB', '2015-01-01', '2018-6-15')
ng_contract = insert_expiration_date('NG', '2015-01-01', '2018-6-15')


In [ ]:
ng_contract.head()

In [ ]:
def plot_forward(df, rows, cols, **kwargs):
    n = rows * cols
    if 'figsize' in kwargs:
        fig = plt.figure(figsize=kwargs['figsize'])
    else:
        fig = plt.figure()
    if 'suptitle' in kwargs:
        fig.suptitle(kwargs['suptitle'])
    ax = [fig.add_subplot(rows, cols, i) for i in range(1, n + 1)]
    for i, ax_ in enumerate(ax):
        ax_.plot(df.iloc[-n + i].values, marker='o')
        ax_.set_title('{:%m-%d}'.format(df.index[-n + i]))
        ax_.axes.set_xticks([])
        ax_.axes.set_yticks([])

def plot_forward_1(df, rows, cols, df_contract, **kwargs):
    n = rows * cols
    if 'figsize' in kwargs:
        fig = plt.figure(figsize=kwargs['figsize'])
    else:
        fig = plt.figure()
    if 'suptitle' in kwargs:
        fig.suptitle(kwargs['suptitle'])
    ax = [fig.add_subplot(rows, cols, i) for i in range(1, n + 1)]
    for i, ax_ in enumerate(ax):
        ax_.plot(df.iloc[-n + i].values, marker='o')
        ax_.set_title('{:%y-%m-%d} ({})'.format(df.index[-n + i], df_contract['remain_date'][-n + i] ))
        ax_.axes.set_xticks([])
        ax_.axes.set_yticks([])        

In [ ]:
#plot_forward(cl, 20, 8, figsize=(14, 22), suptitle='CL')
plot_forward_1(cl, 20, 8, cl_contract, figsize=(14, 22), suptitle='CL')


In [ ]:
plot_forward_1(xb, 20, 8, xb_contract, figsize=(14, 22), suptitle='xb')


In [ ]:
plot_forward_1(ho, 20, 8, ho_contract, figsize=(14, 22), suptitle='ho')

In [ ]:
plot_forward_1(ng, 20, 8, ng_contract, figsize=(14, 22), suptitle='ng')

### 出っ張ったり，凹んだりしているところを狙い撃ちするストラテジー
+ 両側の限月の平均値と比べて，真ん中の限月の価格がずれている時を狙う


In [ ]:
## 例えば，f2 と f4 の平均値が，f3と比べてどのくらいずれているかを調べる

def debeso_checker(data, target1, target2, target3, **kwargs):
    df = data[[target1, target2, target3]]
    df['mean'] = (df[target1] + df[target3])/2
    df['diff'] = (df[target2]/df['mean'])
    df['diff'].hist(bins=100,  alpha=kwargs['alpha'], color=kwargs['color'])
    print(df.describe())

debeso_checker(cl, 'f1','f2','f3', alpha=0.7, color='blue')
debeso_checker(cl, 'f2','f3', 'f4', alpha=0.6, color='green')
debeso_checker(cl, 'f3', 'f4', 'f5', alpha=0.4, color='purple')